In [1]:
import torch
import torchvision
import torch.nn as nn
from model1 import CBAM_ResNet18
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from modelselfv1 import CB_resnet50

In [2]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
train_set = torchvision.datasets.CIFAR10(
    root='../data/CIFAR10/'
    ,train=True
    ,download=False
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip()
        
    ])
)

In [7]:
network = CB_resnet50(num_classes=10).to('cuda')
network.train()

train_loader = torch.utils.data.DataLoader(train_set,batch_size = 12)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

batch = next(iter(train_loader))
images = batch[0].to('cuda')
labels = batch[1].to('cuda')

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()#计算梯度
optimizer.step()#更新权重

#-----------------------
print('loss1:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds,labels)
print('loss2:',loss.item())


loss1: 2.3435921669006348
loss2: 1.9925097227096558


In [5]:
network =  CB_resnet50().to('cuda')
network.train()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=6)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0
count = 0
for batch in train_loader:
    images = batch[0].to('cuda')
    labels = batch[1].to('cuda')
    
    preds = network(images)
    loss = F.cross_entropy(preds,labels)
    #------------要将梯度归零 因为pytorch会累加梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds,labels)
    print('where',count,'loss:',loss.item())
    count +=1
print("epoch:",0,"total_correct",total_correct," loss:",total_loss)
total_correct / len(train_set)

where 0 loss: 6.755741119384766
where 1 loss: 6.9347920417785645
where 2 loss: 6.490289211273193
where 3 loss: 6.568028926849365
where 4 loss: 6.553834438323975
where 5 loss: 6.480274677276611
where 6 loss: 6.23283576965332
where 7 loss: 6.047046661376953
where 8 loss: 5.754188537597656
where 9 loss: 5.3021368980407715
where 10 loss: 5.619600772857666
where 11 loss: 4.897103786468506
where 12 loss: 5.497387409210205
where 13 loss: 5.006518840789795
where 14 loss: 4.41038179397583
where 15 loss: 5.176956653594971
where 16 loss: 4.445199489593506
where 17 loss: 4.51713228225708
where 18 loss: 3.941370725631714
where 19 loss: 4.132331371307373
where 20 loss: 3.99348521232605
where 21 loss: 3.5104312896728516
where 22 loss: 3.8665812015533447
where 23 loss: 3.866610527038574
where 24 loss: 2.8676087856292725
where 25 loss: 3.5733683109283447
where 26 loss: 3.2522308826446533
where 27 loss: 3.307957887649536
where 28 loss: 3.1537158489227295
where 29 loss: 3.7476541996002197
where 30 loss: 

0.3271

In [6]:
network =  resnet34().to('cuda')

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0

for epoch in range(5):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images = batch[0].to('cuda')
        labels = batch[1].to('cuda')
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        #------------要将梯度归零 因为pytorch会累加梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)
total_correct/len(train_set)

epoch: 0 total_correct 18221  loss: 945.8663983345032
epoch: 1 total_correct 24252  loss: 716.9034006595612
epoch: 2 total_correct 27969  loss: 619.8336728215218
epoch: 3 total_correct 30339  loss: 553.833398103714
epoch: 4 total_correct 32261  loss: 505.9548798799515


0.64522

In [6]:
torch.save(network.state_dict(),'pt')